# LangGraph + Text2SQL

## 1. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)


module_path = ".."
add_python_path(module_path)



python path: /home/sagemaker-user/Self-Study-Generative-AI/lab/10_advanced_agentic_text2sql is added
sys.path:  ['/home/sagemaker-user/Self-Study-Generative-AI/lab/10_advanced_agentic_text2sql/lab2_text2sql_langgraph', '/home/sagemaker-user/.conda/envs/text2sql/lib/python310.zip', '/home/sagemaker-user/.conda/envs/text2sql/lib/python3.10', '/home/sagemaker-user/.conda/envs/text2sql/lib/python3.10/lib-dynload', '', '/home/sagemaker-user/.conda/envs/text2sql/lib/python3.10/site-packages', '/home/sagemaker-user/Self-Study-Generative-AI/lab/10_advanced_agentic_text2sql']


### 라이브러리 불러오기

In [2]:
from src.local_utils  import ( 
    converse_with_bedrock,
    init_boto3_client,
    init_search_resources,
    get_column_description,
    search_by_keywords,
    create_prompt,
    NodeTester,
    csv_list_response_format,
    json_response_format,
    # wrapped_bedrock_converse
    converse_with_bedrock_langfuse,
)

import boto3
import json
import copy
from botocore.config import Config

from langchain_aws import BedrockEmbeddings
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from src.opensearch import OpenSearchHybridRetriever, OpenSearchClient
from textwrap import dedent
from langfuse.decorators import observe



### LLM 모델 설정

In [3]:
# llm_model = "us.amazon.nova-pro-v1:0"
# llm_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# llm_model = "us.anthropic.claude-3-5-sonnet-20240620-v1:0"
# llm_model = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
# llm_model = "us.anthropic.claude-3-sonnet-20240229-v1:0"
llm_model = "anthropic.claude-3-sonnet-20240229-v1:0"

### Langfuse 설정

In [4]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv("../.env")

from langfuse.callback import CallbackHandler
import os

langfuse_handler = CallbackHandler(
    public_key=os.environ.get('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.environ.get('LANGFUSE_SECRET_KEY'),
    host=os.environ.get('LANGFUSE_HOST'),
)

# connection test
print("Langfuse 인증 테스트:", langfuse_handler.auth_check())


from langfuse.callback import CallbackHandler
import os

langfuse_handler = CallbackHandler(
    public_key=os.environ.get('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.environ.get('LANGFUSE_SECRET_KEY'),
    host=os.environ.get('LANGFUSE_HOST'),
)

Langfuse 인증 테스트: True


In [5]:
# from langfuse.decorators import observe

# claude_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# # Converesation according to AWS spec including prompting + history
# user_message = """당신은 SmartFinance Advisors 회사가 만든 Alex라는 AI 개인 재무 어드바이저 역할을 수행하게 됩니다. 당신의 목표는 사용자에게 재무 조언과 지침을 제공하는 것입니다. 당신은 SmartFinance Advisors 사이트에 있는 사용자들에게 답변할 것이며, 만약 당신이 Alex의 캐릭터로 응답하지 않으면 사용자들이 혼란스러워할 수 있습니다.
# 다음은 질문 이전의 대화 내역(사용자와 당신 사이)입니다. 대화 내역이 없다면 비어있을 수 있습니다:
# <history>
# User: 안녕하세요 Alex, 당신의 조언이 정말 기대돼요!
# Alex: 안녕하세요! 저는 SmartFinance Advisors의 AI 개인 재무 어드바이저 Alex입니다. 오늘 어떤 재무 목표에 도움이 필요하신가요?
# </history>
# 다음은 상호작용에 있어 중요한 규칙들입니다:

# 항상 SmartFinance Advisors의 AI인 Alex의 캐릭터를 유지하세요.
# 응답 방법이 확실하지 않다면 "죄송합니다, 잘 이해하지 못했습니다. 질문을 다시 말씀해 주시겠어요?"라고 말하세요.
# """
 
# system_messages = "당신은 Finance 전문가 입니다." 
# system_prompts = [
#     {"text": system_messages}
# ]
# conversation = [
#     {
#         "role": "user",
#         "content": [{"text": user_message}],
#     }
# ]
 
# @observe()
# def examples_bedrock_converse_api():
#   responses = {}
 
#   responses["anthropic"] = wrapped_bedrock_converse(
#     modelId=claude_model_id,
#     messages=conversation,
#     system=system_prompts,
#     inferenceConfig={"maxTokens":4096,"temperature": 0.0, "topP": 0.1},
#     additionalModelRequestFields={"top_k":1}
#   )
 
#   return responses
 
# res = examples_bedrock_converse_api()
 
# for key, value in res.items():
#     print(f"{key.title()}\n{value}\n")

### DB Engine 및 오브젝트 정의

In [6]:
engine = create_engine("sqlite:///Chinook.db")
db = SQLDatabase(engine)
DIALECT = "sqlite"

session = boto3.session.Session()
region_name = session.region_name
print(f"현재 리전: {region_name}")

boto3_client = init_boto3_client(region_name)
sql_search_client, table_search_client, sql_retriever, table_retriever = init_search_resources(region_name)



현재 리전: us-east-1


## 2. 검색 기능 테스트

In [7]:
# 검색 키워드 예시
# keyword = "사람들이 좋아하는 장르"
keyword = "2022년 매출 상위 10개 국가는?"

sql_search_result = sql_retriever.invoke(keyword, ensemble=[0.60, 0.40])
# sql_search_result = sql_retriever.invoke(keyword, ensemble=[0.51, 0.49])
table_search_result = table_retriever.invoke(keyword, ensemble=[0.60, 0.40])

if sql_search_result:
    page_content = json.loads(sql_search_result[0].page_content)
    print("샘플 쿼리 검색 결과: ", json.dumps(page_content, indent=4, ensure_ascii=False))

if table_search_result:
    page_content = json.loads(table_search_result[0].page_content)
    print("테이블 검색 결과: ", json.dumps(page_content, indent=4, ensure_ascii=False))

샘플 쿼리 검색 결과:  {
    "input": "고객별 총 구매액 상위 5명의 고객 ID와 총 구매액 조회\n\n상세 목적 및 동기:\n- 고객 구매 패턴 분석을 통해 주요 고객 파악 및 마케팅 전략 수립\n- 최상위 고객에 대한 특별 혜택 제공 또는 맞춤형 프로모션 진행\n- 고객 충성도 제고 및 재구매 유도를 위한 전략 수립\n- 매출 기여도가 높은 고객 관리 강화",
    "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5"
}
테이블 검색 결과:  {
    "table_name": "Invoice",
    "table_summary": "이 테이블은 고객 거래 내역과 관련된 세부 정보를 포함하고 있습니다. 각 송장에 대해 고유 식별자, 관련 고객 ID, 송장 날짜, 청구 주소, 도시, 주, 국가, 우편번호 및 총 금액이 기록됩니다. \n\n이 테이블을 활용하여 다음과 같은 분석이 가능합니다:\n\n- 전체 거래 건수 및 총 매출 규모 파악\n- 고객별 구매 패턴 및 총 구매액 분석\n- 지역별 매출 현황 분석 (청구지 주소 기준)\n- 기간별 매출 추이 분석\n- 주요 고객 파악 및 고객 관계 관리 전략 수립\n- 재고 관리 및 수요 예측\n- 마케팅 및 프로모션 전략 수립\n- 영업 실적 평가 및 성과 분석\n\n이 테이블은 회사의 매출, 고객 관계, 재고 관리, 마케팅 등 다양한 비즈니스 의사결정에 기초 데이터로 활용될 수 있습니다. 고객 구매 패턴, 지역별 수요, 시간별 추이 등을 파악하여 회사의 전략을 수립하고 운영 효율성을 높이는 데 도움이 될 것입니다."
}


In [8]:
# for result in sql_search_result:
#     page_content = json.loads(result.page_content)
#     print("샘플 쿼리 검색 결과: ", json.dumps(page_content, indent=4, ensure_ascii=False))


## 3. LangGraph 

### 그래프 상태 설정

In [9]:
from typing import TypedDict

class GraphState(TypedDict):
    question: str  
    intent: str
    sample_queries: list
    readiness: str
    tables_summaries: list
    table_names: list
    table_details: list
    query_state: dict
    next_action: str
    answer: str
    dialect: str


## 서브그래프1 - 스키마링킹 정의

###  analyze_intent 함수 정의

    


In [10]:
# @observe(name="Analyze Intent Node")
def analyze_intent(state: GraphState) -> GraphState:
    question = state["question"]
    sys_prompt_template = dedent("""당신은 사용자 질문의 의도를 파악하는 비서입니다. 
                                    당신의 임무는 사용자 질문을 하나로 분류하는 것입니다.""")
    usr_prompt_template = dedent("""사용자 질문을 해결하기 위해 데이터베이스 조회가 필요하면 `database` 그 밖의 경우는 `general`로만 응답하세요.
                                    추가 설명이나 이유를 포함하지 마세요. 단어 하나만 정확히 응답하세요.
                                    서두는 생략하세요.
                                    \n\n #질문: {question}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question)
    intent = converse_with_bedrock(sys_prompt, usr_prompt)
    # intent = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)

    return GraphState(intent=intent)

#### 함수 로직 확인



In [11]:
node_tester = NodeTester()
test_state = {
    "question": "2022년 매출 상위 10개 국가는?",
}
result_state = node_tester.test(analyze_intent, test_state)

## Test Result:
{
  "intent": "database"
}


### get_sample_queries 함수 정의

In [12]:
from textwrap import dedent

# @observe(name="get_sample_queries Node")
def get_sample_queries(state: GraphState) -> GraphState:
    question = state["question"]
    samples = sql_retriever.invoke(question, ensemble=[0.80, 0.20])
    page_contents = [doc.page_content for doc in samples if doc is not None]
    sample_inputs = [json.loads(content)['input'] for content in page_contents]

    sys_prompt_template = dedent("""당신은 사용자 질문에 대한 SQL 쿼리를 작성하는 유능한 데이터베이스 엔지니어입니다. 
                          당신의 임무는 질문에 부합한 SQL 쿼리 작성에 유용한 샘플 쿼리를 선택하는 것입니다. 
                          당신이 선택하는 샘플 쿼리는 쿼리 재활용, 스키마 참조 등에 사용될 수 있습니다.""")
    usr_prompt_template = dedent("""질문에 부합한 SQL 쿼리 작성을 위해 유용한 샘플 쿼리를 선택하여 중요도 순서로 정렬하여 응답하세요. 
                                    샘플 쿼리의 인덱스 번호(0부터 시작)만 응답하세요. 관련된 샘플이 없으면 빈 목록("")으로 응답하세요. \n\n 
                                    #질문: {question}\n\n #샘플 쿼리:\n {sample_inputs}\n\n #형식: {csv_list_response_format}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, sample_inputs=sample_inputs, csv_list_response_format=csv_list_response_format)
    sample_ids = converse_with_bedrock(sys_prompt, usr_prompt)
    # sample_ids = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)
    try:
        if sample_ids == '""' or sample_ids.strip() == "":
            return GraphState(sample_queries=[])
        else:
            sample_ids_list = [int(id.strip()) for id in sample_ids.split(',') if id.strip().isdigit()]
            sample_queries = [json.loads(page_contents[id]) for id in sample_ids_list] if sample_ids_list else []
            return GraphState(sample_queries=sample_queries)
    except:
        return GraphState(sample_queries=[])


#### 함수 로직 확인

In [13]:
# def test_node(node_tester, node_function, test_state,result_state ):
#     # test_state와 result_state를 합쳐서 test_state2 생성
#     test_state = {**test_state, **result_state}
#     print("## test_state: ")
#     print(json.dumps(test_state, indent=2, ensure_ascii=False))

#     result_state = node_tester.test(node_function, test_state)
#     return result_state, test_state

# result_state, test_state = test_node(node_tester=node_tester, 
#                             node_function=get_sample_queries, 
#                             test_state=test_state,
#                             result_state=result_state )

### check_readiness 함수 정의

In [14]:
def check_readiness(state: GraphState) -> GraphState:
    question = state["question"]
    sample_queries = state["sample_queries"]
    
    if "table_details" in state:
        table_details = state["table_details"]
    else:
        table_details = []

    sys_prompt_template = dedent("""당신은 사용자 질문에 대한 SQL 쿼리를 작성하는 유능한 데이터베이스 엔지니어입니다. 
                                    당신의 임무는 주어진 DB 정보를 바탕으로, 사용자 질문에 대한 SQL 쿼리 작성이 가능한지 판단하는 것입니다.""")
    usr_prompt_template = dedent("""질문에 대한 SQL 쿼리를 생성하기에 충분한 정보가 제공되었는지 판단합니다. 
                                정보가 충분하면 `Ready`, 정보가 부족하면 `Not Ready`로만  응답하세요. 중요한 건 서두 및 추가 설명은 생략 하세요.\n\n 
                                #질문: {question}\n\n 
                                #샘플 쿼리:\n {sample_queries}\n\n 
                                #사용 가능한 테이블:\n {table_details}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, sample_queries=sample_queries, table_details=table_details)
    # print("usr_prompt: ", json.dumps(usr_prompt, indent=2, ensure_ascii=False))
    readiness = converse_with_bedrock(sys_prompt, usr_prompt)
    # readiness = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)    
    
    return GraphState(readiness=readiness)


In [15]:
# result_state, test_state = test_node(node_tester=node_tester, 
#                             node_function=check_readiness, 
#                             test_state=test_state,
#                             result_state=result_state )


### get_relevant_tables 함수 정의

In [16]:
def get_relevant_tables(state: GraphState) -> GraphState:
    question = state["question"]
    tables = table_retriever.invoke(question, ensemble=[0.40, 0.60])
    page_contents = [doc.page_content for doc in tables if doc is not None]
    table_inputs = [json.loads(content)['table_summary'] for content in page_contents]

    sys_prompt_template = dedent("""당신은 사용자 요청에 맞는 SQL 쿼리를 작성하는 유능한 데이터베이스 엔지니어입니다. 
                                당신의 임무는 SQL 쿼리 작성에 필요한 테이블을 선택하는 것입니다.""")
    usr_prompt_template = dedent("""사용자 요청에 맞는 SQL 쿼리를 생성하기 위해 필요한 테이블을 선택하여, 
                                이를 중요도 순서로 정렬한 후 인덱스 번호(0부터 시작)로 응답하세요. 사용자 요청에 관련된 테이블이 없으면 빈 목록("")으로 응답하세요.\n\n 
                                #질문: {question}\n\n 
                                #테이블 정보:\n {table_inputs}\n\n 
                                #형식: {csv_list_response_format}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, table_inputs=table_inputs, csv_list_response_format=csv_list_response_format)
    table_ids = converse_with_bedrock(sys_prompt, usr_prompt)
    # table_ids = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)        
    try:
        if table_ids == '""' or table_ids.strip() == "":
            return GraphState(tables=[], table_names=[])
        else:
            table_ids_list = [int(id.strip()) for id in table_ids.split(',') if id.strip().isdigit()]
            tables = [json.loads(page_contents[id]) for id in table_ids_list] if table_ids_list else []
            table_names = [table['table_name'] for table in tables]
            return GraphState(tables=tables, table_names=table_names)
    except:
        return GraphState(tables=[], table_names=[])


In [17]:
# result_state, test_state = test_node(node_tester=node_tester, 
#                             node_function=get_relevant_tables, 
#                             test_state=test_state,
#                             result_state=result_state )


### describe_schema 함수 정의

In [18]:
def describe_schema(state: GraphState) -> GraphState:
    table_names = state["table_names"]
    table_details = []
    data = db.get_table_info_no_throw(table_names)
    
    if not isinstance(data, list):
        data = [data]
    
    for item in data:
        if isinstance(item, str):
            items = item.split('CREATE TABLE')
        else:
            items = [item]
        
        for i in range(1, len(items)):
            sub_item = 'CREATE TABLE' + items[i]
            for table_name in table_names:
                if f'CREATE TABLE "{table_name}"' in sub_item:
                    
                    parts = sub_item.split('/*', 1)
                    sql_statement = parts[0].strip()
                    
                    sample_data = "No sample data available"
                    if len(parts) > 1:
                        sample_part = parts[1].split('*/', 1)[0] 
                        sample_lines = sample_part.strip().split('\n')
                        if len(sample_lines) > 1:
                            sample_data = '\n'.join(sample_lines)
                    
                    table_desc = get_column_description(table_name) if table_search_client else {}        
                    table_detail = {
                        "table": table_name,
                        "cols": table_desc if table_desc else {},
                        "create_table_sql": sql_statement,
                        "sample_data": sample_data
                    }
                    
                    if not table_detail["cols"]:
                        print(f"No columns found for table {table_name}")
                    table_details.append(table_detail)
                    
    return GraphState(table_details=table_details)


In [19]:
# result_state, test_state = test_node(node_tester=node_tester, 
#                             node_function=describe_schema, 
#                             test_state=test_state,
#                             result_state=result_state )


In [20]:


def next_step_by_intent(state: GraphState) -> GraphState:
    return state["intent"]

def next_step_by_readiness(state: GraphState) -> GraphState:
    return state["readiness"]



## 서브그래프2 - 쿼리 생성/검증/실행/재시도

### generate_query 함수 정의

In [21]:
### Query Generation & Execution - SubGraph2

initial_query_state = {
    "status": "success",
    "query": "",
    "result": "",
    "error": {
        "code": "",
        "message": "",
        "failed_step": "",
        "hint": ""
    }
}


dialect = DIALECT
def generate_query(state: GraphState) -> GraphState:
    new_query_state = copy.deepcopy(initial_query_state)
    question = state["question"]
    sample_queries = state["sample_queries"]
    table_details = state["table_details"]

    query_state = state.get("query_state", {}) or {}
    error_info = query_state.get("error", {}) or {}
    hint = error_info.get("hint", "None")
    
    sys_prompt_template = dedent("""당신은 사용자 질문에 대한 {dialect} SQL 쿼리를 작성하는 유능한 데이터베이스 엔지니어입니다. 
                                당신의 임무는 주어진 DB 정보를 바탕으로, 사용자 질문에 부합하는 정확한 SQL 쿼리를 작성하는 것입니다.""")
    usr_prompt_template = dedent("""다음 주어진 샘플 쿼리 및 스키마 정보, 과거 실패 이력을 바탕으로 DB dialect에 맞는 쿼리를 작성합니다. 
                                서두는 생략하고 생성된 SQL 쿼리 구문만 제공하세요. \n\n 
                                #질문: {question}\n\n 
                                #샘플 쿼리:\n {sample_queries}\n\n 
                                #사용 가능한 테이블:\n {table_details}\n\n, 
                                #추가 정보 (과거 실패 이력, 추가 획득한 정보 등):\n {hint}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, dialect=dialect, sample_queries=sample_queries, table_details=table_details, hint=hint)
    generated_query = converse_with_bedrock(sys_prompt, usr_prompt)
    # generated_query = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)        

    new_query_state["query"] = generated_query

    return GraphState(query_state=new_query_state)


In [22]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=generate_query, 
#                             test_state=test_state,
#                             result_state=result_state )


### validate_query 함수 정의

In [23]:
def validate_query(state: GraphState) -> GraphState:
    dialect = DIALECT
    question = state["question"]
    query_state = copy.deepcopy(state["query_state"])
    query = query_state["query"]
    
    explain_statements = {
        'mysql': "EXPLAIN {query}",
        'mariadb': "EXPLAIN {query}",
        'sqlite': "EXPLAIN QUERY PLAN {query}",
        'oracle': "EXPLAIN PLAN FOR\n{query}\n\nSELECT * FROM TABLE(DBMS_XPLAN.DISPLAY);",
        'postgresql': "EXPLAIN ANALYZE {query}",
        'postgres': "EXPLAIN ANALYZE {query}",
        'presto': "EXPLAIN ANALYZE {query}",
        'sqlserver': "SET STATISTICS PROFILE ON; {query}; SET STATISTICS PROFILE OFF;"
    }
    
    if dialect.lower() not in explain_statements:
        query_plan = " "
    else:
        try:
            explain_query = explain_statements[dialect.lower()].format(query=query)
            query_plan = db.run(explain_query)
        except Exception as e:
            query_state["status"] = "error"
            query_state["error"]["code"] = "E01"
            query_state["error"]["message"] = f"An error occurred while executing the EXPLAIN query: {str(e)}"
            query_state["error"]["failed_step"] = "validation"
            query_state["query"] = query
            return GraphState(query_state=query_state)

    sys_prompt_template = dedent("""당신은 사용자 질문에 대한 기존 {dialect} SQL 쿼리를 검토하고, 
                                필요 시 최적화하는 데이터베이스 전문가입니다. 
                                당신의 임무는 주어진 SQL 쿼리 및 추가 정보를 바탕으로 쿼리의 정합성, 
                                최적화 가능성을 검토하고, 이에 입각한 최종 쿼리를 제공하는 것입니다. """)
    usr_prompt_template = dedent("""사용자의 질문에 맞춰 쿼리에 alias를 추가해주세요. 
                                기존 SQL 쿼리에서 사용되지 않았던 테이블, 컬럼을 추가하는 것은 허용되지 않습니다. 
                                서두는 생략하고 생성된 SQL 쿼리 구문만 제공하세요. \n\n 
                                #질문: {question}\n\n 
                                #기존 쿼리:\n {query}\n\n 
                                #쿼리 플랜:\n {query_plan}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, dialect=dialect, query=query, query_plan=query_plan)
    validated_query = converse_with_bedrock(sys_prompt, usr_prompt)
    # validated_query = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)            
    query_state["query"] = validated_query

    return GraphState(query_state=query_state)

In [24]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=validate_query, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### execute_query 함수 정의

In [25]:
def execute_query(state: GraphState) -> GraphState:
    query_state = copy.deepcopy(state["query_state"])
    query = query_state["query"]
    try:
        query_state["result"] = db.run(query)
    except Exception as e:
        query_state["status"] = "error"
        query_state["error"]["code"] = "E02"
        query_state["error"]["message"] = f"An error occurred while executing the validated query: {str(e)}"
        query_state["error"]["failed_step"] = "execution"
        return GraphState(query_state=query_state)
    return GraphState(query_state=query_state)

In [26]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=execute_query, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### handle_failure 함수 정의

In [27]:
def handle_failure(state: GraphState) -> GraphState:
    query_state = copy.deepcopy(state["query_state"])
    query = query_state['query']
    message = query_state['error']['message']
    sys_prompt_template = dedent("""당신은 SQL 쿼리의 실패를 처리하는 유능한 데이터베이스 엔지니어입니다. 
                                당신의 임무는 주어진 SQL 쿼리의 실패 원인을 파악하여, 문제 해결을 위한 다음 작업을 결정하는 것입니다.""")
    usr_prompt_template = dedent("""주어진 SQL 쿼리의 실패 메시지를 바탕으로 다음 중 하나의 원인(`failure_type`)과 해결을 위한 실마리(`hint`)를 함께 제공합니다.\n 
                                다음은 failure_type의 선택 예시입니다.\n
                                부정확한 쿼리 구문 작성: `syntax_check`\n
                                스키마 불일치: `schema_check`\n
                                DB 외부요인(권한, 연결 문제 등): `stop`\n
                                DB의 일시적 오동작(쿼리 재실행 필요): `retry`\n\n
                                #실패 쿼리: {query}\n\n
                                #실패 메시지: {message}\n\n#형식: {json_response_format}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, query=query, message=message, json_response_format=json_response_format)
    # result = converse_with_bedrock(sys_prompt, usr_prompt)
    result = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)            
    json_result = json.loads(result)

    query_state["hint"] = json_result["hint"]
    return GraphState(next_action=json_result["failure_type"], query_state=query_state)


In [28]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=handle_failure, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### get_relevant_columns 함수 정의

In [29]:

def get_relevant_columns(state: GraphState) -> GraphState:
    query_state = copy.deepcopy(state["query_state"])
    question = state["question"]
    query = query_state["query"]
    message = query_state['error']['message']
    sys_prompt_template = dedent("""당신은 SQL 쿼리의 실패를 처리하는 유능한 데이터베이스 엔지니어입니다. 
                                당신의 임무는 앞서 발생한 쿼리 실패를 해결하기 위해 스키마를 재탐색 하는 것입니다.""")
    usr_prompt_template = dedent("""사용자 질문과 주어진 실패 메시지를 바탕으로 쿼리 재탐색에 적절한 키워드를 제공하세요. 
                                서두는 생략하고 키워드만 형식에 맞춰 응답하세요. \n\n 
                                #사용자 질문: {question}\n\n
                                #이전 실패 쿼리: {query}\n\n
                                메시지: {message}\n\n
                                #형식: {csv_list_response_format}""")
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, query=query, message=message, csv_list_response_format=csv_list_response_format)
    # keywords = converse_with_bedrock(sys_prompt, usr_prompt)
    keywords = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)            
    return keywords

def next_step_by_query_state(state:GraphState) -> GraphState:
    return state["query_state"]["status"]

def next_step_by_next_action(state:GraphState) -> GraphState:
    return state["next_action"]


In [30]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=get_relevant_columns, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### get_general_answer 함수 정의

In [31]:
def get_general_answer(state: GraphState) -> GraphState:
    question = state["question"]
    sys_prompt_template = dedent("""당신은 사용자의 일반적인 질문에 답변하는 유능한 비서입니다. 
                                    잘 모르는 주제에 대한 질문에 대해서는 모른다고 답변하세요.""")
    usr_prompt_template = "#질문: {question}"
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question)
    answer = converse_with_bedrock(sys_prompt, usr_prompt)
    # answer = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)            

    return GraphState(answer=answer)


In [32]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=get_general_answer, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### get_database_answer 함수 정의

In [33]:

def get_database_answer(state: GraphState) -> GraphState:
    question = state["question"]
    query_state = state["query_state"]
    query = query_state["query"]
    data = query_state["result"]
    failed_step = query_state["error"]["failed_step"]
    message = query_state["error"]["message"]
    sys_prompt_template = dedent("""당신은 데이터베이스의 정보를 바탕으로 사용자의 질문에 답변하는 
                                    유능한 비서입니다. 당신의 임무는 주어진 참고정보를 참고하여, 
                                    사용자의 질문에 성실히 답변하는 것입니다.""")
    if query_state["status"] == "success":
        usr_prompt_template = dedent("""답변에는 사용된 쿼리, 데이터프레임(Markdown Table), 
                                    질문에 대한 간단한 답변을 포함해야 합니다. \n\n
                                    #질문: {question}\n\n
                                    #사용된 쿼리: {query}\n\n
                                    #데이터: {data}\n\n""")
        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, query=query, data=data)
    else:
        usr_prompt_template = dedent("""다음에는 사용자 질문에 대한 쿼리 수행에 실패한 기록이 주어집니다. 
                                        이를 바탕으로 요청 처리에 실패한 이유를 설명하세요.\n\n
                                        #질문: {question}\n\n
                                        #사용된 쿼리: {query}\n\n
                                        #실패 단계: {failed_step}\n\n
                                        #에러메시지: {message}\n\n""")
        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, query=query, failed_step=failed_step, message=message)
    
    answer = converse_with_bedrock(sys_prompt, usr_prompt)
    # answer = converse_with_bedrock_langfuse(sys_prompt, usr_prompt, model_id=llm_model)            
    return GraphState(answer=answer)


In [34]:
# result_state2, test_state2 = test_node(node_tester=node_tester, 
#                             node_function=get_database_answer, 
#                             test_state=test_state2,
#                             result_state=result_state2 )

### LangGraph 워크플로 그래프 생성 

In [35]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(GraphState)

# Global Nodes
workflow.add_node("analyze_intent", analyze_intent)
workflow.add_node("get_general_answer", get_general_answer)
workflow.add_node("get_database_answer", get_database_answer)
workflow.set_entry_point("analyze_intent")

# SubGraph1 Nodes - Schema Linking
workflow.add_node("get_sample_queries", get_sample_queries)
workflow.add_node("check_readiness", check_readiness)
workflow.add_node("get_relevant_tables", get_relevant_tables)
workflow.add_node("describe_schema", describe_schema)

# SubGraph2 Nodes - Query Generation & Execution
workflow.add_node("generate_query", generate_query)
workflow.add_node("validate_query", validate_query)
workflow.add_node("execute_query", execute_query)
workflow.add_node("handle_failure", handle_failure)
workflow.add_node("get_relevant_columns", get_relevant_columns)

# Edge from Entry to SubGraph1
workflow.add_conditional_edges(
    "analyze_intent",
    next_step_by_intent,
    {
        "database": "get_sample_queries",
        "general": "get_general_answer",
    }
)

# Edges in SubGraph1
workflow.add_edge("get_sample_queries", "check_readiness")
workflow.add_conditional_edges(
    "check_readiness"    ,
    next_step_by_readiness,
    {
        "Ready": "generate_query",
        "Not Ready": "get_relevant_tables"
    }
)
workflow.add_edge("get_relevant_tables", "describe_schema")
workflow.add_edge("describe_schema", "check_readiness")

# Edges in SubGraph2
workflow.add_edge("generate_query", "validate_query")
workflow.add_conditional_edges(
    "validate_query"    ,
    next_step_by_query_state,
    {
        "success": "execute_query",
        "error": "handle_failure"
    }
)
workflow.add_conditional_edges(
    "execute_query"    ,
    next_step_by_query_state,
    {
        "success": "get_database_answer",
        "error": "handle_failure"
    }
)
workflow.add_conditional_edges(
    "handle_failure"    ,
    next_step_by_next_action,
    {
        "schema_check": "get_relevant_columns",
        "syntax_check": "generate_query",
        "retry": "validate_query",
        "stop": "get_database_answer"
    }
)
workflow.add_edge("get_relevant_columns", "generate_query")

# Edges to END
workflow.add_edge("get_general_answer", END)
workflow.add_edge("get_database_answer", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### LangGraph 워크플로 그래프 시각화

In [36]:
# from IPython.display import Image, display

# try:
#     display(
#         Image(app.get_graph(xray=True).draw_mermaid_png())
#     )  # 실행 가능한 객체의 그래프를 mermaid 형식의 PNG로 그려서 표시합니다. 
#        # xray=True는 추가적인 세부 정보를 포함합니다.
# except:
#     # 이 부분은 추가적인 의존성이 필요하며 선택적으로 실행됩니다.
#     print("Exception occurred")
#     pass

### LangGraph 워크플로 End-to-End 테스트 (draft)

In [37]:
import pprint
from langgraph.errors import GraphRecursionError
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
            recursion_limit=100, 
            configurable={"thread_id": "TODO"},
            callbacks=[langfuse_handler]  # 여기에 Langfuse 핸들러 추가}
        )
inputs = GraphState(question="2022년 매출 상위 10개 국가는?")

pp = pprint.PrettyPrinter(width=200, compact=True)

try:
    for output in app.stream(inputs, config=config):
        for key, value in output.items():
            print(f"\n🔹 [NODE] {key}")
            print("=" * 80)
            for k, v in value.items():
                print(f"📌 {k}:")
                pp.pprint(v)
            print("=" * 80)
except GraphRecursionError as e:
    print(f"⚠️ Recursion limit reached: {e}")


🔹 [NODE] analyze_intent
📌 intent:
'database'

🔹 [NODE] get_sample_queries
📌 sample_queries:
[{'input': '캐나다에 거주하는 고객 정보 조회. 국가별 고객 데이터 분석 및 마케팅 전략 수립을 위한 데이터 추출이 목적일 수 있습니다. 캐나다 시장 진출 계획 수립 시 현지 고객 기반을 파악하고 타겟 고객층을 정의하는 데 활용될 수 있습니다.',
  'query': "SELECT * FROM Customer WHERE Country = 'Canada'"},
 {'input': '고객별 총 구매액 상위 5명의 고객 ID와 총 구매액 조회\n\n상세 목적 및 동기:\n- 고객 구매 패턴 분석을 통해 주요 고객 파악 및 마케팅 전략 수립\n- 최상위 고객에 대한 특별 혜택 제공 또는 맞춤형 프로모션 진행\n- 고객 충성도 제고 및 재구매 유도를 위한 전략 수립\n- 매출 기여도가 높은 고객 관리 강화',
  'query': 'SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5'},
 {'input': '전체 트랙의 총 재생시간 확인 요청. 음원 서비스 운영을 위해 전체 콘텐츠의 총 길이를 파악하여 스토리지 용량 산정 및 스트리밍 대역폭 예측에 활용할 수 있음.', 'query': 'SELECT SUM(Milliseconds) FROM Track'},
 {'input': "앨범 데이터에서 아티스트 이름이 'AC/DC'인 아티스트의 앨범 정보 조회. 아티스트 테이블에서 'AC/DC'라는 이름을 가진 아티스트의 ArtistId를 찾아 Album 테이블에서 해당 ArtistId와 연결된 모든 앨범 정보를 가져오는 요청. 아티스트 이름으로 특정 아티스트의 앨범 목록을 확인하고자 하는 목적으로 보임.",
  'query': "SELECT * FROM